In [76]:
import pandas as pd

In [77]:
df = pd.read_csv('data/gemstone.csv')

In [78]:
df.head()

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [79]:
df.shape

(193573, 11)

In [80]:
df =df.drop(['id'], axis = 1)

In [81]:
df.head()

,carat,cut,color,clarity,depth,table,x,y,z,price
0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


## dividing the data into dependent and independent features

In [82]:
from sklearn.model_selection import train_test_split

In [83]:
X = df.drop(labels=['price'], axis = 1)
y = df['price']

In [84]:
X.head()

,carat,cut,color,clarity,depth,table,x,y,z
0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55
1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05
2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50
3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71
4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77


In [85]:
y.head()

0    13619
1    13387
2     2772
3      666
4    14453
Name: price, dtype: int64

## dividing into numerical and categorical data

In [86]:
type(X)

pandas.core.frame.DataFrame

In [87]:
categorical_column = X.columns[X.dtypes == 'object']

In [88]:
numerical_column = X.columns[X.dtypes != 'object']

In [89]:
print(categorical_column)
print(numerical_column)

Index(['cut', 'color', 'clarity'], dtype='object')
Index(['carat', 'depth', 'table', 'x', 'y', 'z'], dtype='object')


In [90]:
from sklearn.impute import SimpleImputer #null handling
from sklearn.preprocessing import OrdinalEncoder #for ordinal encoding of categorical columns
from sklearn.preprocessing import StandardScaler # for scaling of numerical data 

## pipelines

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [91]:
print('cut : ', X['cut'].unique())
print('color : ', X['color'].unique())
print('clarity : ', X['clarity'].unique())

cut :  ['Premium' 'Very Good' 'Ideal' 'Good' 'Fair']
color :  ['F' 'J' 'G' 'E' 'D' 'H' 'I']
clarity :  ['VS2' 'SI2' 'VS1' 'SI1' 'IF' 'VVS2' 'VVS1' 'I1']


In [92]:
cut_categories = ['Fair','Good', 'Very Good', 'Premium', 'Ideal']
color_categories = ['D','E','F','G','H','I','J']
clarity_categories = ['I1','SI2','SI1', 'VS2', 'VS1', 'VVS2', 'VVS1','IF']

## Numerical Preprocessing Pipeline

In [100]:
numerical_pipeline = Pipeline(
    steps=
    [
        ('imputer', SimpleImputer(strategy = 'median')),
        ('scaler', StandardScaler())
    ]
)

## Categorical Preprocessing Pipeline

In [102]:
categorical_pipeline = Pipeline(
    steps =
    [
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('encoder', OrdinalEncoder(categories=[cut_categories,color_categories,clarity_categories])),
         ('scaler', StandardScaler())
    ]
)

In [103]:
processor = ColumnTransformer(
    [
        ('numerical_pipeline', numerical_pipeline, numerical_column),
        ('categorical_pipeline', categorical_pipeline, categorical_column)
    ]
)

In [96]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.30, random_state=30)

In [97]:
X_train.shape

(135501, 9)

In [98]:
X_test.shape

(58072, 9)

In [105]:
X_train = pd.DataFrame(processor.fit_transform(X_train),columns=processor.get_feature_names_out())

In [106]:
X_test = pd.DataFrame(processor.transform(X_test), columns = processor.get_feature_names_out())

In [108]:
X_train.head()

,numerical_pipeline__carat,numerical_pipeline__depth,numerical_pipeline__table,numerical_pipeline__x,numerical_pipeline__y,numerical_pipeline__z,categorical_pipeline__cut,categorical_pipeline__color,categorical_pipeline__clarity
0,-0.975439,-0.849607,-0.121531,-1.042757,-1.080970,-1.123150,0.874076,1.528722,1.352731
1,0.235195,1.833637,-0.121531,0.318447,0.279859,0.485354,-2.144558,-0.935071,-0.646786
2,0.494617,0.815855,0.399800,0.570855,0.606458,0.673737,-0.132136,0.296826,0.686225
3,-1.018676,0.260701,0.921131,-1.214034,-1.244270,-1.195605,-0.132136,0.296826,0.019720
4,-0.953821,-0.664555,-0.642862,-1.069801,-1.044681,-1.094168,0.874076,2.144670,1.352731


In [109]:
from sklearn.linear_model import LinearRegression,ridge_regression,Lasso,ElasticNet
from sklearn.metrics import r2_score,mean_squared_error

In [110]:
regressor1 = LinearRegression()
regressor1.fit(X_train,y_train)

LinearRegression()

In [111]:
regressor1.coef_

array([ 6433.66003594,  -132.75843566,   -70.42922179, -1720.30971463,
        -499.29302619,   -63.39317848,    72.44537247,  -460.41604642,
         650.76431652])

In [112]:
regressor1.intercept_

3970.7662895476774

In [113]:
y_pred = regressor1.predict(X_test)

In [114]:
r2_score(y_test,y_pred)

0.9368908248567511